In [20]:
import os
from collections import defaultdict
from random import shuffle

import xml.etree.ElementTree as ET
from tqdm import tqdm
import pandas as pd
import pybboxes as pbx
import shutil

In [21]:
import warnings

warnings.filterwarnings("ignore")

In [22]:
from ultralytics import YOLO

<h1> Pre-processing data


Chosen dataset: https://www.kaggle.com/datasets/biancaferreira/african-wildlife <br>
With this dataset I have 4 classes: buffalo, elephant, rhino, zebra. <br>
Each class have image + annotation. First step I need to divide the dataset into processed_data folder


In [23]:
data_path = "./data/"
processed_data_path = "./processed_data"
os.makedirs(processed_data_path, exist_ok=True)

In [24]:
labels_folder = os.listdir(data_path)

In [25]:
df = pd.DataFrame(
    columns=["class", "img_path", "x_min", "x_max", "y_min", "y_max"]
)

In [26]:
images_destination_folder = f"{processed_data_path}/images/"
labels_destination_folder = f"{processed_data_path}/labels/"
os.makedirs(images_destination_folder, exist_ok=True)
os.makedirs(labels_destination_folder, exist_ok=True)

In [27]:
for label in tqdm(labels_folder):
    org_path = f"{data_path}/{label}"
    files = zip(os.listdir(org_path)[::2], os.listdir(org_path)[1::2])
    for img, file in tqdm(files):
        lb_file = f"{org_path}/{file}"
        img_path = f"{org_path}/{img}"
        numOfImage = len(os.listdir(images_destination_folder))
        img_new_path = os.path.join(
            images_destination_folder, f"africa{numOfImage}.jpg"
        )
        label_new_path = os.path.join(
            labels_destination_folder, f"africa{numOfImage}.txt"
        )
        shutil.copy(
            img_path,
            img_new_path,
        )
        shutil.copy(
            lb_file,
            label_new_path,
        )
        lb_info = open(lb_file, "r").read().split()
        objects = [[] for i in range(0, len(lb_info), 5)]
        for idx, obj_info in enumerate(lb_info):
            if idx % 5 == 0:
                continue
            else:
                objects[idx // 5].append(obj_info)
        for obj_list in objects:
            if obj_list:
                obj_details = obj_list

                lb = label
                x_min = float(obj_details[0])
                x_max = float(obj_details[1])
                y_min = float(obj_details[2])
                y_max = float(obj_details[3])

                row = {
                    "class": lb,
                    "img_path": img_new_path,
                    "x_min": x_min,
                    "x_max": x_max,
                    "y_min": y_min,
                    "y_max": y_max,
                }

                df = df.append(row, ignore_index=True)

376it [00:01, 279.91it/s]00<?, ?it/s]
376it [00:01, 221.05it/s]01<00:04,  1.35s/it]
376it [00:01, 211.86it/s]03<00:03,  1.55s/it]
376it [00:02, 155.58it/s]04<00:01,  1.66s/it]
100%|██████████| 4/4 [00:07<00:00,  1.82s/it]


In [28]:
df

,class,img_path,x_min,x_max,y_min,y_max
0,buffalo,./processed_data/images/africa0.jpg,0.560000,0.663017,0.617500,0.644769
1,buffalo,./processed_data/images/africa1.jpg,0.473515,0.508434,0.497592,0.838554
2,buffalo,./processed_data/images/africa2.jpg,0.819167,0.593750,0.148333,0.242500
3,buffalo,./processed_data/images/africa2.jpg,0.747500,0.472500,0.221667,0.190000
4,buffalo,./processed_data/images/africa2.jpg,0.524167,0.543750,0.165000,0.232500
...,...,...,...,...,...,...
2680,zebra,./processed_data/images/africa1501.jpg,0.433594,0.518919,0.387500,0.656757
2681,zebra,./processed_data/images/africa1502.jpg,0.548828,0.492568,0.330469,0.974324
2682,zebra,./processed_data/images/africa1503.jpg,0.205859,0.592568,0.221094,0.520270
2683,zebra,./processed_data/images/africa1503.jpg,0.431641,0.597297,0.242969,0.562162


In [29]:
train_dir = "./processed_data/train/"
val_dir = "./processed_data/val"
labels_path = "./processed_data/labels/"

In [30]:
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(train_dir + "/images", exist_ok=True)
os.makedirs(train_dir + "/labels", exist_ok=True)
os.makedirs(val_dir + "/images", exist_ok=True)
os.makedirs(val_dir + "/labels", exist_ok=True)

In [31]:
files = os.listdir(images_destination_folder)
shuffle(files)

In [32]:
def split(files, ratio):
    elements = len(files)
    middle = int(elements * ratio)
    return [files[:middle], files[middle:]]


def copy_files(images_path, labels_path, destination_path, files):
    for file_name in files:
        file_name = file_name.split(".")[0]

        src = images_path + f"{file_name}.jpg"
        dst = destination_path + "/images"
        shutil.copy(src, dst)

        src = labels_path + f"{file_name}.txt"
        dst = destination_path + "/labels"
        shutil.copy(src, dst)

In [33]:
train_ratio = 0.75
train_files, val_files = split(files, train_ratio)

copy_files(images_destination_folder, labels_path, train_dir, train_files)
copy_files(images_destination_folder, labels_path, val_dir, val_files)

In [34]:
class_training = {idx: label for idx, label in enumerate(labels_folder)}

In [35]:
class_training

{0: 'buffalo', 1: 'elephant', 2: 'rhino', 3: 'zebra'}

In [36]:
with open(f"./processed_data/africa_data.yaml", "w") as f:
    f.write("train: ./train/images\n")
    f.write("val: ./val/images\n")
    f.write("nc: 4\n")
    f.write(f"names: {class_training}")

In [37]:
model = YOLO("yolov8n.pt")

100%|██████████| 6.23M/6.23M [00:00<00:00, 51.6MB/s]


In [38]:
results = model.train(
    data="./processed_data/africa_data.yaml", epochs=10, imgsz=320
)  # train the model
results = model.val()  # evaluate model performance on the validation set

Ultralytics YOLOv8.0.233 🚀 Python-3.9.13 torch-2.1.0+cpu CPU (AMD Ryzen 7 6800H with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=./processed_data/africa_data.yaml, epochs=10, time=None, patience=50, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, sh

train: Scanning C:\Users\ADMIN\OneDrive - EPITA\Computer-Vision-DSA-23\Lab02\processed_data\train\labels... 1128 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1128/1128 [00:01<00:00, 933.51it/s]

train: WARNING ⚠️ C:\Users\ADMIN\OneDrive - EPITA\Computer-Vision-DSA-23\Lab02\processed_data\train\images\africa53.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ C:\Users\ADMIN\OneDrive - EPITA\Computer-Vision-DSA-23\Lab02\processed_data\train\images\africa784.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ C:\Users\ADMIN\OneDrive - EPITA\Computer-Vision-DSA-23\Lab02\processed_data\train\images\africa788.jpg: corrupt JPEG restored and saved


train: New cache created: C:\Users\ADMIN\OneDrive - EPITA\Computer-Vision-DSA-23\Lab02\processed_data\train\labels.cache


val: Scanning C:\Users\ADMIN\OneDrive - EPITA\Computer-Vision-DSA-23\Lab02\processed_data\val\labels... 376 images, 0 backgrounds, 0 corrupt: 100%|██████████| 376/376 [00:00<00:00, 940.81it/s]

...

                   all        376        676       0.89      0.827      0.917      0.732
               buffalo        376        136       0.94      0.816      0.935        0.8
              elephant        376        186      0.841      0.844      0.903      0.694
                 rhino        376        144      0.903      0.837      0.934      0.774
                 zebra        376        210      0.875       0.81      0.895      0.658
Speed: 0.2ms preprocess, 11.2ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs\detect\train2
Ultralytics YOLOv8.0.233 🚀 Python-3.9.13 torch-2.1.0+cpu CPU (AMD Ryzen 7 6800H with Radeon Graphics)
Model summary (fused): 168 layers, 3006428 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\ADMIN\OneDrive - EPITA\Computer-Vision-DSA-23\Lab02\processed_data\val\labels.cache... 376 images, 0 backgrounds, 0 corrupt: 100%|██████████| 376/376 [00:00<?, ?it/s]

val: WARNING ⚠️ C:\Users\ADMIN\OneDrive - EPITA\Computer-Vision-DSA-23\Lab02\processed_data\val\images\africa1438.jpg: corrupt JPEG restored and saved



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [00:08<00:00,  2.79it/s]


                   all        376        676      0.888      0.831      0.918      0.732
               buffalo        376        136       0.94      0.824      0.937        0.8
              elephant        376        186      0.853      0.844      0.905      0.698
                 rhino        376        144      0.897      0.845      0.936      0.776
                 zebra        376        210      0.865       0.81      0.892      0.656
Speed: 0.2ms preprocess, 10.5ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs\detect\train22


In [39]:
model.export(format='onnx')

Ultralytics YOLOv8.0.233 🚀 Python-3.9.13 torch-2.1.0+cpu CPU (AMD Ryzen 7 6800H with Radeon Graphics)

PyTorch: starting from 'runs\detect\train2\weights\best.pt' with input shape (1, 3, 320, 320) BCHW and output shape(s) (1, 8, 2100) (5.9 MB)
requirements: Ultralytics requirement ['onnx>=1.12.0'] not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 14.3/14.3 MB 25.2 MB/s eta 0:00:00
   ---------------------------------------- 413.4/413.4 kB ? eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6

requirements: AutoUpdate success ✅ 23.0s, installed 1 package: ['onnx>=1.12.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.15.0 opset 17...
ONNX: export success ✅ 23.6s, saved as 'runs\detect\train2\weights

'runs\\detect\\train2\\weights\\best.onnx'